In [19]:
from typing import List


class Solution:

    def handleQuery(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        n = len(nums1)
        nums1 = int(''.join(map(str, nums1[::-1])), 2)
        s = sum(nums2)
        ans = []
        for op, l, r in queries:
            if op == 1:
                d = r - l + 1
                x = (1 << d) - 1
                x <<= l
                nums1 ^= x
            elif op == 2:
                s += l * nums1.bit_count()
            else:
                ans.append(s)
        return ans

1

In [1]:
from typing import List


class Node:

    def __init__(self, si, ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.lgh = ei - si + 1
        self.left = self.right = None
        self.val = 0  # 区间1的个数
        self.lazy = False  # 懒标记：待办更新


class SegmentTree:

    def __init__(self, si, ei):
        self.root = Node(si, ei)

    def push_down(self, node: Node):  # 动态开点及下发懒标记
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy:
            node.left.val = node.left.lgh - node.left.val
            node.right.val = node.right.lgh - node.right.val
            node.left.lazy = not node.left.lazy
            node.right.lazy = not node.right.lazy
            node.lazy = False

    def update(self, node: Node, l, r):  # 对 node与[l,r]交集的节点进行01取反
        if l <= node.si and node.ei <= r:
            node.val = node.lgh - node.val
            node.lazy = not node.lazy
            return
        self.push_down(node)
        if node.mi >= l:
            self.update(node.left, l, r)
        if node.mi + 1 <= r:
            self.update(node.right, l, r)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, l, r):
        if l <= node.si and node.ei <= r:
            return node.val
        self.push_down(node)
        ans = 0
        if node.mi >= l:
            ans += self.query(node.left, l, r)
        if node.mi + 1 <= r:
            ans += self.query(node.right, l, r)
        return ans


class Solution:

    def handleQuery(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        n = len(nums1)
        st = SegmentTree(0, n - 1)
        for i, num in enumerate(nums1):
            if num:
                st.update(st.root, i, i)
        ans = []
        s = sum(nums2)
        for op, l, r in queries:
            if op == 1:
                st.update(st.root, l, r)
            elif op == 2:
                s += l * st.query(st.root, 0, n - 1)
            else:
                ans.append(s)
        return ans

![](2023-03-15-18-06-40.png)

In [2]:
class Solution:

    def handleQuery(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        n = len(nums1)
        cnt1 = [0] * (4 * n)  # 节点值：区间1的个数
        todo = [False] * (4 * n)  # 是否代办节点（懒标记） 该节点已经更新 子节点未更新

        def maintain(o):  # 维护节点
            cnt1[o] = cnt1[o * 2] + cnt1[o * 2 + 1]

        def do(o, l, r):  # 代办处理（懒标记下发）
            cnt1[o] = r - l + 1 - cnt1[o]
            todo[o] = not todo[o]

        def push_down(o, l, r, m):
            if todo[o]:
                do(o * 2, l, m)
                do(o * 2 + 1, m + 1, r)
                todo[o] = False

        def build(o, l, r):  # 初始化线段树
            if l == r:
                cnt1[o] = nums1[l]
                return
            m = l + r >> 1
            build(o * 2, l, m)
            build(o * 2 + 1, m + 1, r)
            maintain(o)

        def update(o, l, r, L, R):  # 翻转节点[L,R]
            if L <= l and r <= R:
                do(o, l, r)
                return
            m = l + r >> 1
            push_down(o, l, r, m)
            if m >= L:
                update(o * 2, l, m, L, R)
            if m + 1 <= R:
                update(o * 2 + 1, m + 1, r, L, R)
            maintain(o)

        build(1, 0, n - 1)
        ans = []
        s = sum(nums2)
        for op, l, r in queries:
            if op == 1:
                update(1, 0, n - 1, l, r)
            elif op == 2:
                s += l * cnt1[1]
            else:
                ans.append(s)
        return ans


In [ ]:
class Node:
    def __init__(self,si,ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.lgh = ei - si + 1
        self.left = self.right = None
        self.val = 0 # 区间1的个数
        self.lazy = False # 懒标记：待办更新

class SegmentTree:
    def __init__(self,si,ei):
        self.root = Node(si,ei)
    def push_down(self,node:Node): # 动态开点及下发懒标记
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy:
            node.left.val = node.left.lgh - node.left.val
            node.right.val = node.right.lgh - node.right.val
            node.left.lazy = not node.left.lazy
            node.right.lazy = not node.right.lazy
            node.lazy = False
    def update(self,node:Node,l,r): # 对 node与[l,r]交集的节点进行01取反
        if l <= node.si and node.ei <= r:
            node.val = node.lgh - node.val
            node.lazy = not node.lazy
            return
        self.push_down(node)
        if node.mi >= l:
            self.update(node.left,l,r)
        if node.mi + 1 <= r:
            self.update(node.right,l,r)
        node.val = node.left.val + node.right.val

class Solution:
    def handleQuery(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        n = len(nums1)
        st = SegmentTree(0,n-1)
        for i,num in enumerate(nums1):
            if num:
                st.update(st.root,i,i)
        ans = []
        s = sum(nums2)
        for op,l,r in queries:
            if op == 1:
                st.update(st.root, l, r)
            elif op == 2:
                s += l * st.root.val
            else:
                ans.append(s)
        return ans